# Create DL3 files from DL2 observed data with the `lstchain_create_dl3_file` Tool

To check the simple usage of the Tool, one can do the following in a terminal (`--help-all` for more detail)
```
$ lstchain_create_dl3_file --help
```


Currently, the Tool requires the information of the main observed source, name and RA/Dec position in degrees, to be passed as arguments, to enter source provenance information, in the final event list HDUs.

One should use the same config file for event selection on the observed data, as used on the MC DL2 files for generating the IRFs, which are to be included in the final DL3 file.

If one wants to use energy-dependent cuts on gammaness, the IRFs should also be produced with energy-dependent cuts, and should have the `GH_CUTS` HDU stored with the information of the cuts. The DL3 Tool will use this HDU to implement the specific cuts in the respective reco energy bin. Otherwise, the Tool will look for the global gammaness cut used in the header values of the IRF HDUs, and apply the same global gammaness cut.

In this example, we will use:
 - a DL2 file stored at `/fefs/aswg/workspace/analysis-school-2024/DL2_to_DL3/DL2/`
 - IRF files in `/fefs/aswg/workspace/analysis-school-2024/DL2_to_DL3/IRF` produced previously in each of the MC testing nodes.
 - **IRF interpolation** method

To get more explanation on the data format, check https://gamma-astro-data-formats.readthedocs.io/en/latest/index.html

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from traitlets.config.loader import Config
from pathlib import Path

from astropy.io import fits
from astropy.table import QTable
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.data import EventList

from lstchain.paths import dl2_to_dl3_filename
from lstchain.io.config import read_configuration_file

# Input parameters to convert DL2 file to DL3

In [7]:
# Modify the paths as applicable.
base_dir = Path("/fefs/aswg/workspace/analysis-school-2024/DL2_to_DL3")

input_dl2_path = base_dir / "DL2" / "dl2_LST-1.Run16074.h5"
irf_directory = base_dir / "IRF"

dl3_dir = "DL2_to_DL3/DL3"

dl3_filename = dl2_to_dl3_filename(input_dl2_path)

# output_dl3_path = base_dir / "DL3"
output_dl3_path = Path("./DL3")
output_dl3_path.mkdir(exist_ok=True, parents=True)

# One can used the same config file as the one used for the IRF creation since 
# it also contains the DL3 cuts and event filter sections.
# Vary the following parameters for different cuts.
# config_file = "../docs/examples/irf_tool_config.json"
config_file = "/fefs/aswg/workspace/analysis-school-2024/cta-lstchain/docs/examples/irf_tool_config.json"

source_name = "Crab"
source_ra = "83.63308333deg"
source_dec = "22.0145deg"

# Coordinates can be retrieved from Astropy for sources present in the CDS
# SkyCoord.from_name("Crab")

In [20]:
!ls ./DL3

DataReductionFITSWriter.provenance.log	final_irf.fits.gz
FITSIndexWriter.provenance.log		hdu-index.fits.gz
dl3_LST-1.Run16074.fits			obs-index.fits.gz


In [4]:
config = Config(read_configuration_file(config_file))
config

{'EventSelector': {'filters': {'intensity': [50, inf],
   'width': [0, inf],
   'length': [0, inf],
   'r': [0, 1],
   'wl': [0.01, 1],
   'leakage_intensity_width_2': [0, 1],
   'event_type': [32, 32]}},
 'DL3Cuts': {'min_event_p_en_bin': 100,
  'global_gh_cut': 0.7,
  'gh_efficiency': 0.7,
  'min_gh_cut': 0.1,
  'max_gh_cut': 0.98,
  'global_alpha_cut': 10,
  'global_theta_cut': 0.2,
  'theta_containment': 0.7,
  'alpha_containment': 0.7,
  'min_theta_cut': 0.1,
  'max_theta_cut': 0.32,
  'fill_theta_cut': 0.32,
  'min_alpha_cut': 1,
  'max_alpha_cut': 20,
  'fill_alpha_cut': 20,
  'allowed_tels': [1]},
 'DataBinning': {'true_energy_min': 0.005,
  'true_energy_max': 500,
  'true_energy_n_bins': 25,
  'reco_energy_min': 0.005,
  'reco_energy_max': 500,
  'reco_energy_n_bins': 25,
  'energy_migration_min': 0.2,
  'energy_migration_max': 5,
  'energy_migration_n_bins': 30,
  'fov_offset_min': 0.1,
  'fov_offset_max': 1.1,
  'fov_offset_n_edges': 9,
  'bkg_fov_offset_min': 0,
  'bkg_fov_

!lstchain_create_dl3_file --help

# Running the tool

In [10]:
!lstchain_create_dl3_file \
--input-dl2 $input_dl2_path \
--input-irf-path $irf_directory \
--output-dl3-path $output_dl3_path \
--source-name $source_name \
--source-ra=$source_ra \
--source-dec=$source_dec \
--config $config_file \
--overwrite

The metadata are comparable
The other parameter axes data are comparable
2024-02-15 14:53:15,399 ERROR [lstchain.high_level.interpolate] (interpolate.interpolate_irf): AL CUTS not present for IRF interpolation
/fefs/aswg/software/conda/envs/lst-school-2024-02/lib/python3.11/site-packages/lstchain/reco/utils.py:536: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


# Produce the index files

 - hdu-index.fits.gz
 - obs-index.fits.gz

!lstchain_create_dl3_index_files --help-all

In [11]:
! lstchain_create_dl3_index_files \
--input-dl3-dir $output_dl3_path \
--file-pattern dl3*fits \
--overwrite

2024-02-15 14:56:34,278 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 16074 does not contain HDU BACKGROUND
2024-02-15 14:56:34,278 ERROR [lstchain.high_level.hdu_table] (hdu_table.create_hdu_index_hdu): Run 16074 does not contain HDU PSF


# Check the DL3 info

In [12]:
f = fits.open(output_dl3_path / dl3_filename)
f.info()

Filename: DL3/dl3_LST-1.Run16074.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  EVENTS        1 BinTableHDU     76   2006989R x 11C   [K, D, D, D, D, D, K, D, D, D, D]   
  2  GTI           1 BinTableHDU     29   1R x 2C   [D, D]   
  3  POINTING      1 BinTableHDU     44   1R x 5C   [D, D, D, D, D]   
  4  EFFECTIVE AREA    1 BinTableHDU     46   1R x 5C   [25D, 25D, D, D, 25D]   
  5  ENERGY DISPERSION    1 BinTableHDU     52   1R x 7C   [25D, 25D, 30D, 30D, D, D, 750D]   
  6  GH_CUTS       1 BinTableHDU     35   25R x 5C   [D, D, D, K, D]   
  7  RAD_MAX       1 BinTableHDU     49   1R x 5C   [25D, 25D, D, D, 25D]   


In [13]:
QTable.read(output_dl3_path / "hdu-index.fits.gz")

OBS_ID,HDU_TYPE,HDU_CLASS,FILE_DIR,FILE_NAME,HDU_NAME,SIZE
int64,bytes8,bytes10,bytes1,bytes23,bytes17,int64
16074,events,events,.,dl3_LST-1.Run16074.fits,EVENTS,176679360
16074,gti,gti,.,dl3_LST-1.Run16074.fits,GTI,176679360
16074,pointing,pointing,.,dl3_LST-1.Run16074.fits,POINTING,176679360
16074,aeff,aeff_2d,.,dl3_LST-1.Run16074.fits,EFFECTIVE AREA,176679360
16074,edisp,edisp_2d,.,dl3_LST-1.Run16074.fits,ENERGY DISPERSION,176679360
16074,rad_max,rad_max_2d,.,dl3_LST-1.Run16074.fits,RAD_MAX,176679360


In [14]:
QTable.read(output_dl3_path / "hdu-index.fits.gz").meta

OrderedDict([('CREATOR', 'lstchain v0.10.7'),
             ('HDUDOC',
              'https://github.com/open-gamma-ray-astro/gamma-astro-data-formats'),
             ('HDUVERS', '0.3'),
             ('HDUCLASS', 'GADF'),
             ('ORIGIN', 'CTA'),
             ('TELESCOP', 'CTA-N'),
             ('CREATED', '2024-02-15 14:56:34.280'),
             ('HDUCLAS1', 'INDEX'),
             ('HDUCLAS2', 'HDU'),
             ('INSTRUME', 'LST-1'),
             ('BASE_DIR',
              '/fefs/home/francesco.schiavone/lst-analysis-school-2024/notebooks/DL3'),
             ('EXTNAME', 'HDU INDEX')])

In [15]:
QTable.read(output_dl3_path / "obs-index.fits.gz")

OBS_ID,DATE-OBS,TIME-OBS,DATE-END,TIME-END,RA_PNT,DEC_PNT,ZEN_PNT,ALT_PNT,AZ_PNT,RA_OBJ,DEC_OBJ,TSTART,TSTOP,ONTIME,TELAPSE,LIVETIME,DEADC,OBJECT,OBS_MODE,N_TELS,TELLIST,INSTRUME
,,,,,deg,deg,deg,deg,deg,deg,deg,s,s,s,s,s,,,,,,
int64,bytes10,bytes12,bytes10,bytes12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,bytes8,int64,bytes5,bytes5
16074,2023-12-14,03:11:29.897,2023-12-14,03:30:59.252,83.34511831068029,21.80115499165838,29.046791625038964,60.95320837496104,263.3361856730758,83.63308333,22.0145,164171489.89745498,164172659.25241733,1169.355238199234,1169.354962348938,1112.4332065035733,0.9513218653868446,Crab,POINTING,1,LST-1,LST-1


In [16]:
QTable.read(output_dl3_path / "obs-index.fits.gz").meta

OrderedDict([('CREATOR', 'lstchain v0.10.7'),
             ('HDUDOC',
              'https://github.com/open-gamma-ray-astro/gamma-astro-data-formats'),
             ('HDUVERS', '0.3'),
             ('HDUCLASS', 'GADF'),
             ('ORIGIN', 'CTA'),
             ('TELESCOP', 'CTA-N'),
             ('CREATED', '2024-02-15 14:56:34.308'),
             ('HDUCLAS1', 'INDEX'),
             ('HDUCLAS2', 'OBS'),
             ('INSTRUME', 'LST-1'),
             ('MJDREFI', 58392),
             ('MJDREFF', 0.0),
             ('EXTNAME', 'OBS INDEX')])

In [17]:
# Look into the DL3 content
filename = output_dl3_path / "dl3_LST-1.Run16074.fits"
events = EventList.read(filename)

In [18]:
events.table[:5]

EVENT_ID,TIME,RA,DEC,ENERGY,GAMMANESS,MULTIP,GLON,GLAT,ALT,AZ
,s,deg,deg,TeV,,,deg,deg,deg,deg
int64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
20,164171489.89745498,83.546712726762,21.07714821185084,0.039865483559792976,0.5155527139901388,1,185.31064235793858,-6.353690632302413,63.019661084757885,260.0244000792608
23,164171489.89751577,84.18216877648118,22.093525561969894,0.06167756963619449,0.3899505776676602,1,184.7631696865144,-5.3117981969133,63.94291972537065,261.65480279724505
41,164171489.89779043,80.82580090683412,21.251005014912273,0.047126796347841844,0.4769775173269689,1,183.77357659507047,-8.38961213364383,60.7292657046114,262.3717792149874
47,164171489.89787507,83.71632895612436,20.68860192769277,0.06177167310212579,0.46515517411858204,1,185.72637563474638,-6.42710157568462,63.01519608256777,259.099600877809
52,164171489.89793444,84.66801515231819,21.66871028656808,0.03143057008245397,0.48073873981822934,1,185.36491064080428,-5.154799112465871,64.2095619925816,260.37580587028896
